🌍 Dust maps from GEOS-Chem TXT files (step-by-step)

This script reads GEOS-Chem text files containing
longitude, latitude, and dust concentration,
and creates one map per file.

In [53]:
#Import libraries

import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import cartopy.crs as ccrs
import cartopy.feature as cfeature


In [55]:
# USER INPUT: paths to folders and files

# CHANGE THESE PATHS TO YOUR OWN

# ================= USER-DEFINED PATHS =================

# Folder with GEOS-Chem TXT files
input_folder = "/Users/julia/GEOS chem/geos-chem_txt_files"

# Folder where output images will be saved
output_folder = "/Users/julia/GEOS chem/geos-chem dust events/output_folder"

# Create output folder if it does not exist
os.makedirs(output_folder, exist_ok=True)


In [57]:
# Function to read one GEOS-Chem TXT file

def read_geoschem_txt(file_path):
    """
    Reads a GEOS-Chem TXT file and converts it to 2D grids.

    Parameters
    ----------
    file_path : str
        Full path to the TXT file.

    Returns
    -------
    lon_grid : 2D numpy array
        Longitude grid.
    lat_grid : 2D numpy array
        Latitude grid.
    dust_grid : 2D numpy array
        Dust concentration on the grid.
    """

    # Load numerical data from file
    data = np.loadtxt(file_path)

    # Extract columns
    lon = data[:, 0]
    lat = data[:, 1]
    dust = data[:, 2]

    # Get unique longitude and latitude values
    lon_unique = np.unique(lon)
    lat_unique = np.unique(lat)

    # Create 2D coordinate grids
    lon_grid, lat_grid = np.meshgrid(lon_unique, lat_unique)

    # Reshape concentration to 2D grid
    dust_grid = dust.reshape(len(lat_unique), len(lon_unique))

    return lon_grid, lat_grid, dust_grid


In [59]:
# Create a colormap 

def lighten_cmap(cmap_name, factor=0.85):
    """
    Lightens an existing matplotlib colormap by mixing it with white.

    Parameters
    ----------
    cmap_name : str
        Name of the matplotlib colormap.
    factor : float
        0 = white, 1 = original colormap.

    Returns
    -------
    new_cmap : matplotlib colormap
    """

    base_cmap = plt.get_cmap(cmap_name)
    colors = base_cmap(np.linspace(0, 1, 256))
    white = np.ones((256, 4))

    new_colors = colors * factor + white * (1 - factor)

    return mcolors.ListedColormap(new_colors)


# Lightened colormap for dust
light_cmap = lighten_cmap("Spectral_r", factor=0.85)



In [61]:
# Loop over all TXT files and create maps

for filename in sorted(os.listdir(input_folder)):

    # Process only TXT files
    if not filename.endswith(".txt"):
        continue

    # Full path to the file
    file_path = os.path.join(input_folder, filename)

    # ---------------------------------------------------
    # Extract date from file name (YYYY_MM_DD.txt)
    # ---------------------------------------------------
    date_str = filename.replace(".txt", "")
    formatted_date = date_str.replace("_", "-")

    # ---------------------------------------------------
    # Read GEOS-Chem data
    # ---------------------------------------------------
    lon, lat, dust = read_geoschem_txt(file_path)

    # ---------------------------------------------------
    # Create map
    # ---------------------------------------------------
    fig, ax = plt.subplots(
        figsize=(12, 6),
        subplot_kw={"projection": ccrs.PlateCarree()}
    )

    # Plot dust concentration
    pcm = ax.pcolormesh(
        lon, lat, dust,
        cmap=light_cmap,
        vmin=0,
        vmax=30,
        transform=ccrs.PlateCarree()
    )

    # Colorbar
    cbar = plt.colorbar(pcm, ax=ax)
    cbar.set_label(r'Dust (μg m$^{-3}$)', fontsize=18)
    cbar.ax.tick_params(labelsize=15)

    # Add map features
    ax.add_feature(cfeature.BORDERS, linewidth=1)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)

    # Focus on Asia
    ax.set_extent([70, 150, 15, 55], crs=ccrs.PlateCarree())

    # Gridlines
    gl = ax.gridlines(draw_labels=True, linewidth=0.5,
                      color='gray', alpha=0.5)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'fontsize': 17}
    gl.ylabel_style = {'fontsize': 17}

    # Title with date
    ax.set_title(
        f'Dust Area Over Asia (Ground Level) – {formatted_date}',
        fontsize=20
    )

    # ---------------------------------------------------
    # Save figure to file
    # ---------------------------------------------------
    output_file = f"dust_map_{date_str}.png"
    save_path = os.path.join(output_folder, output_file)

    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close()

# =====================================================
# Final message after ALL files have been processed
# =====================================================

print("DONE AND SAVED ✅")
print("All maps have been saved to the following folder:")
print(output_folder)



DONE AND SAVED ✅
All maps have been saved to the following folder:
/Users/julia/GEOS chem/geos-chem dust events/output_folder
